In [ ]:
!python -m spacy download en_trf_bertbaseuncased_lg

In [1]:
import spacy

nlp = spacy.load("en_trf_bertbaseuncased_lg")
#doc = nlp("Apple shares rose on the news. Apple pie is delicious.")
#print(doc[0].similarity(doc[7]))
#print(doc._.trf_last_hidden_state.shape)

# an example of spacy preserved some word embedding

In [2]:
doc1 = nlp(u'they did an awesome job')
doc2 = nlp(u'they did an awful job')
doc3 = nlp(u'they did a great job')
 
print (doc1.similarity(doc2)) 
print (doc2.similarity(doc3)) 
print (doc1.similarity(doc3)) 

0.9134093016230975
0.9053232267859165
0.9737446020998256


In [3]:
doc2.vector.shape

(768,)

In [6]:
from scipy.spatial.distance import cosine
cosine(doc1.vector,doc2.vector)

0.0865907073020935

# Vectorize the texts : step 1
## Steps 
1. vectorize every piece of reviews(3k docs) in dictionary doc_vecs , key is review_id, value is the vector 
2. compute similarity and store them (3k * 3k) in distance_matrix 
3. For every month, mkt, find out how many reviews are out-there and retreive the pairwise distances. So if there are 10 reviews, need to retrieve 90 values 

## Input: raw reviews texts from /0_data/Lock_ES_RawData/installer_review_data_20180410.csv

In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import os

reviews=pd.read_csv('../0_data/Lock_ES_RawData/installer_review_data_20180410.csv',sep=',',escapechar='\\')


In [7]:
import time
from collections import defaultdict
doc_vecs = []
reviews=reviews[['id','body']].dropna()
start_time = time.time()
for i, row in reviews.iterrows():
    doc_vecs.append([row.id,nlp(row['body']).vector])
    if i%100==0:
        elapsed_time = time.time() - start_time
        print('we vectorized {} reviews and time has elapsed {}'.format(i,elapsed_time))

import pandas as pd 
reviews_doc_vecs_df=pd.DataFrame(data=doc_vecs,columns=['reviewid','vectorized_text'])
temp_df_doc_vecs=reviews_doc_vecs_df

from scipy.spatial.distance import cosine
from itertools import combinations 

start_time = time.time()
distances_list=[]
counter=0
for item in combinations(temp_df_doc_vecs.reviewid,2):
    #print(item)
    #print(cosine(temp_df_doc_vecs.iloc[item[0]-1].vectorized_text,temp_df_doc_vecs.iloc[item[1]-1].vectorized_text))
    vec1=temp_df_doc_vecs[temp_df_doc_vecs['reviewid']==item[0]].vectorized_text.tolist();
    vec2=temp_df_doc_vecs[temp_df_doc_vecs['reviewid']==item[1]].vectorized_text.tolist();
    d=cosine(vec1,vec2)
    data_collect=[item[0],item[1],d]
    distances_list.append(data_collect)  
    counter=counter+1      
    if counter%100000==0:
        elapsed_time = time.time() - start_time
        print('we have processed {} pairs and time has elapsed {}'.format(counter,elapsed_time))
        distances_df=pd.DataFrame(data=distances_list,columns=['vec_review_id_1','vec_review_id_2','cosine_distance'])
        filename='BERT_distances_pairwire_dec30'+str(counter/100000)+'.csv'
        distances_df.to_csv(filename)
        distances_list=[]
        

we vectorized 0 reviews and time has elapsed 0.3196842670440674
we vectorized 100 reviews and time has elapsed 60.20851683616638
we vectorized 200 reviews and time has elapsed 100.01848268508911
we vectorized 300 reviews and time has elapsed 139.13074707984924
we vectorized 400 reviews and time has elapsed 174.80279302597046
we vectorized 500 reviews and time has elapsed 212.08138346672058
we vectorized 600 reviews and time has elapsed 252.14534330368042
we vectorized 700 reviews and time has elapsed 288.0042989253998
we vectorized 800 reviews and time has elapsed 326.5471487045288
we vectorized 900 reviews and time has elapsed 371.2351825237274
we vectorized 1000 reviews and time has elapsed 406.4341835975647
we vectorized 1100 reviews and time has elapsed 449.0208420753479
we vectorized 1200 reviews and time has elapsed 491.01585602760315
we vectorized 1300 reviews and time has elapsed 520.635381937027
we vectorized 1400 reviews and time has elapsed 562.695565700531
we vectorized 150

In [8]:
import pandas as pd
import glob
distances_pairs=pd.DataFrame()
for f in glob.glob('BERT_distances_pairwire_dec30*.csv'):
    df = pd.read_csv(f)
    distances_pairs=pd.concat([df,distances_pairs])
    
    

## Output: pairwise distances of all pairs  '../3_output/ALL_BERT_distances_pairwise_dec30.csv')

In [9]:
distances_pairs.to_csv('../3_output/ALL_BERT_distances_pairwise_dec30.csv')